In [16]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus

client = QdrantClient(host="localhost", port=6333)
client

In [17]:
# Let's create our first collection

my_collection = "first_collection"

first_collection = client.recreate_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)
print(first_collection)

True


C:\Users\Admin\AppData\Local\Temp\ipykernel_18168\895106094.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  first_collection = client.recreate_collection(


In [18]:
collection_info = client.get_collection(collection_name=my_collection)
list(collection_info)

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', None),
 ('indexed_vectors_count', 0),
 ('points_count', 0),
 ('segments_count', 8),
 ('config',
  CollectionConfig(params=CollectionParams(vectors=VectorParams(size=100, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), qu

In [19]:
assert collection_info.status == CollectionStatus.GREEN
assert collection_info.vectors_count == None

# Adding points

In [20]:
import numpy as np

In [21]:
data = np.random.uniform(low=-1.0, high=1.0, size=(1_000, 100))
type(data[0, 0]), data[:2, :20], len(data[0])

(numpy.float64,
 array([[ 0.71244486, -0.50107936,  0.12391711, -0.4517674 , -0.69019078,
          0.79937544,  0.14699948,  0.09496133, -0.65802617,  0.29755816,
          0.74537842,  0.56492299,  0.92635142, -0.84504117, -0.55715399,
         -0.01498292,  0.13322574, -0.27071789, -0.52762162,  0.78240913],
        [-0.42836013,  0.40353968, -0.82514205, -0.43050681,  0.80399298,
          0.74842065, -0.23003923, -0.55834748, -0.45542012, -0.67748674,
         -0.39835459, -0.05760136, -0.82121022, -0.29297294, -0.78482415,
          0.21734489,  0.72562375, -0.52893372,  0.36432622, -0.48219598]]),
 100)

In [22]:
index = list(range(len(data)))
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

In [23]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:
client.retrieve(
    collection_name=my_collection,
    ids=[100],
    with_vectors=True # the default is False
)

[Record(id=100, payload={}, vector=[-0.025133813, -0.106980145, 0.14892611, 0.011296607, 0.10271142, 0.094712034, -0.13835452, -0.15864046, -0.02622933, -0.11880851, 0.030190885, 0.028164558, -0.16372004, -0.18040568, 0.17232695, -0.061910424, -0.0012277651, -0.06298824, 0.07028838, -0.022066537, -0.14195219, 0.026955362, 0.10088724, 0.063020594, 0.0826999, -0.020799577, 0.11952137, 0.061504725, -0.063839085, 0.11425105, 0.17082217, -0.15450312, -0.10211645, 0.059065275, 0.047303163, -0.026399296, -0.1591974, -0.07376001, 0.014031549, -0.016406812, 0.14387935, 0.075568765, -0.04003736, -0.14071606, 0.022911012, 0.07171648, -0.09659579, -0.051002927, 0.12590568, 0.08411448, 0.036869075, -0.18063681, -0.14987484, -0.076828726, 0.051165815, 0.07265047, 0.11277705, -0.110590614, -0.15470491, -0.041771907, -0.07485027, -0.172346, -0.034819797, -0.12862542, 0.108487196, -0.0712643, -0.12983328, -0.07600198, 0.091493234, -0.03629605, -0.15963806, 0.05366608, -0.03263417, -0.066000365, -0.1006

In [28]:
# update the collection one point at a time

def create_song():
    return np.random.uniform(low=-1.0, high=1.0, size=100).tolist()

client.upsert(
    collection_name=my_collection,
    points=[
        models.PointStruct(
            id=1000,
            vector=create_song(),
        )
    ]
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [29]:
# We can also delete a point in a straightforward fashion.

# this will show the amount of vectors BEFORE deleting the one we just created
print(client.count(
    collection_name=my_collection, 
    exact=True,
))

client.delete(
    collection_name=my_collection,
    points_selector=models.PointIdsList(
        points=[1000],
    ),
)

print(client.count(
    collection_name=my_collection, 
    exact=True,
))

count=1001
count=1000
